In [1]:
%matplotlib widget

In [2]:
import matplotlib.pyplot as plt

import ipywidgets as ipw
from widget_periodictable import PTableWidget

from IPython.display import display

from aiidalab_sssp.inspect.subwidgets.select import SSSP_JSON, SSSP_ARCHIVE
from aiidalab_sssp.inspect.subwidgets import plot as wplot
from aiidalab_sssp.inspect.subwidgets.select import PseudoSelectWidget
from aiidalab_sssp.inspect.subwidgets.summary import SummaryWidget

In [ ]:
plt.ioff()
# Show the PTable widget
periodic_table = PTableWidget(states = 1,
                      selected_colors = ['royalblue'],
                      unselected_color='cornflowerblue', border_color = 'black', width = '40px')

disable_elements = [e for e in periodic_table.allElements if e not in SSSP_JSON.keys()]
periodic_table.disabled_elements = disable_elements

last_selected = periodic_table.selected_elements
def on_element_select(event):
    global last_selected

    if event['name'] == 'selected_elements' and event['type'] == 'change':
        if tuple(event['new'].keys()) == ('Du', ):
            last_selected = event['old']
        elif tuple(event['old'].keys()) == ('Du', ):
            #print(last_selected, event['new'])
            if len(event['new']) != 1:
                # Reset to only one element only if there is more than one selected,
                # to avoid infinite loops
                newly_selected = set(event['new']).difference(last_selected)
                # If this is empty it's ok, unselect all
                # If there is more than one, that's weird... to avoid problems, anyway, I pick one of the two
                if newly_selected:
                    periodic_table.selected_elements = {list(newly_selected)[0]: 0}
                else:
                    periodic_table.selected_elements = {}
                # To have the correct 'last' value for next calls
                last_selected = periodic_table.selected_elements

periodic_table.observe(on_element_select)

display(periodic_table)

In [ ]:
pseudo_select_widget = PseudoSelectWidget()

ipw.dlink(
    (periodic_table, 'selected_elements'), 
    (pseudo_select_widget, 'selected_element'),
    transform=lambda elements: list(elements.keys())[0] if elements else None,
)

display(pseudo_select_widget)

In [ ]:
# Check if the page load with a finished verification
from aiida.orm import load_node, Dict
from aiida.common import AttributeDict
import aiida
from urllib.parse import parse_qs, urlsplit
from aiida.tools.importexport import ExportFileFormat, export

aiida.load_profile('default')

def flatten_output(attr_dict, node_collection):
    """
    flaten output dict node
    node_collection is a list to accumulate the nodes that not unfolded

    For some namespace do not unfold dict
    - band_parameters

    For output nodes not being expanded, write down the uuid for making archive
    """
    do_not_unfold = ["band_parameters", "scf_parameters", "seekpath_parameters"]

    for key, value in attr_dict.items():
        if isinstance(value, AttributeDict):
            # keep on unfold
            flatten_output(value, node_collection)
        elif isinstance(value, Dict) and key not in do_not_unfold:
            attr_dict[key] = value.get_dict()
        else:
            # node type not handled attach uuid
            attr_dict[key] = value.uuid
            node_collection.append(value.uuid)

    # print(archive_uuids)
    return attr_dict



def parse_url(url):
    query = parse_qs(urlsplit(url).query)
    pk = query.get('pk', None)

    # Can be a list with multiple nodes to load
    return pk

pk = parse_url(jupyter_notebook_url)
if pk:
    try:
        node = load_node(pk=int(pk[0]))
    except:
        # can not load pk
        raise

    if node.attributes['process_label'] == 'VerificationWorkChain':
        res = {}
        archive_uuids = []

        label = node.extras.get("label")
        attr_dict = node.outputs._construct_attribute_dict(incoming=False)

        res[label] = flatten_output(attr_dict, archive_uuids)

        import json

        res = json.loads(json.dumps(dict(res), indent=2))

        # once the selected_element set the selected_pseudos will be updated
        pseudo_select_widget.selected_element = res[label]["pseudo_info"]["element"]

        res.update(pseudo_select_widget.pseudos_dict)
        pseudo_select_widget.pseudos_dict = res

        # kwargs = {
        #     "input_calc_forward": False,
        #     "input_work_forward": False,
        #     "create_backward": False,
        #     "return_backward": False,
        #     "call_calc_backward": False,
        #     "call_work_backward": False,
        #     "include_comments": False,
        #     "include_logs": False,
        #     "overwrite": True,
        # }

        # # print(archive_uuids)
        # export(
        #     [load_node(i) for i in archive_uuids],
        #     filename=archive,
        #     file_format=ExportFileFormat.ZIP,
        #     **kwargs
        # )

In [ ]:
from aiidalab_sssp.inspect.subwidgets.bands import BandsWidget

bands_widget = BandsWidget()
ipw.dlink((pseudo_select_widget, 'selected_pseudos'), (bands_widget, 'selected_pseudos'))

display(bands_widget)

In [ ]:
from aiidalab_sssp.inspect.subwidgets.delta import EOSwidget

eos_widget = EOSwidget()
ipw.dlink((pseudo_select_widget, 'selected_pseudos'), (eos_widget, 'selected_pseudos'))

display(eos_widget)

In [ ]:
plot_delta_measure_widget = wplot.PlotDeltaMeasureWidget()
ipw.dlink((pseudo_select_widget, 'selected_pseudos'), (plot_delta_measure_widget, 'selected_pseudos'))

plot_cohesive_energy_converge_widget = wplot.PlotCohesiveEnergyConvergeWidget()
ipw.dlink((pseudo_select_widget, 'selected_pseudos'), (plot_cohesive_energy_converge_widget, 'selected_pseudos'))

plot_cohesive_energy_converge_diff_widget = wplot.PlotCohesiveEnergyConvergeDiffWidget()
ipw.dlink((pseudo_select_widget, 'selected_pseudos'), (plot_cohesive_energy_converge_diff_widget, 'selected_pseudos'))

plot_phonon_frequencies_converge_abs_widget = wplot.PlotPhononFrequenciesConvergeAbsWidget()
ipw.dlink((pseudo_select_widget, 'selected_pseudos'), (plot_phonon_frequencies_converge_abs_widget, 'selected_pseudos'))

plot_phonon_frequencies_converge_rel_widget = wplot.PlotPhononFrequenciesConvergeRelWidget()
ipw.dlink((pseudo_select_widget, 'selected_pseudos'), (plot_phonon_frequencies_converge_rel_widget, 'selected_pseudos'))

plot_pressure_converge_widget = wplot.PlotPressureConvergeWidget()
ipw.dlink((pseudo_select_widget, 'selected_pseudos'), (plot_pressure_converge_widget, 'selected_pseudos'))

plot_pressure_converge_rel_widget = wplot.PlotPressureConvergeRelWidget()
ipw.dlink((pseudo_select_widget, 'selected_pseudos'), (plot_pressure_converge_rel_widget, 'selected_pseudos'))

plot_delta_converge_widget = wplot.PlotDeltaConvergeWidget()
ipw.dlink((pseudo_select_widget, 'selected_pseudos'), (plot_delta_converge_widget, 'selected_pseudos'))

plot_delta_converge_rel_widget = wplot.PlotDeltaConvergeRelWidget()
ipw.dlink((pseudo_select_widget, 'selected_pseudos'), (plot_delta_converge_rel_widget, 'selected_pseudos'))

plot_bands_converge_widget = wplot.PlotBandsConvergeWidget()
ipw.dlink((pseudo_select_widget, 'selected_pseudos'), (plot_bands_converge_widget, 'selected_pseudos'))

pass

## $\Delta$ measures and $\nu$-factor measure

Please check <a href="./sssp-docs.ipynb" target="_blank">Documentation</a> for more details.

In [ ]:
display(plot_delta_measure_widget)

## Summary of convergence verification 

The convergence criteria we use here is:
- Cohesive energy: $E^{abs}_{coh} < 2.0 (\text{meV/atom})$
- Phonon frequencies: $\delta \bar{\omega} < 1.0 (\%)$
- Pressure: $\delta P < 0.5 (\%)$

In [ ]:
summary_widget = SummaryWidget()
ipw.dlink((pseudo_select_widget, 'selected_pseudos'), (summary_widget, 'selected_pseudos'))
display(summary_widget)

#### ⚠️ Notes:
For wavefunction cutoff convergence verification, the charge density cutoff is fixed at $x_c^{rho}=200\times d$, where $d=4$ for norm-converserving pseudopotentials and $d=8$ for ultra-soft and paw pseudopotentials. 

For charge density cutoff convergence verification, the wavefunction cutoff is fixed at converged value of wavefunction cutoff convergence test.

The reference cutoff $(x_c^{wfc}, x_c^{rho})$ is set to $(200, 200\times d)$ for both wavefunctian and charge density verifications.


## Cohesive energy (eV/atom)

Please check <a href="./sssp-docs.ipynb" target="_blank">Documentation</a> for more details.

In [ ]:
display(plot_cohesive_energy_converge_widget)

## Cohesive energy error (eV/atom)

Please check [Documentation](./sssp-docs.ipynb) for more details.

In [ ]:
display(plot_cohesive_energy_converge_diff_widget)

## Phonon frequencies absolute error ($cm^{-1}$)

Please check <a href="./sssp-docs.ipynb" target="_blank">Documentation</a> for more details.

In [ ]:
display(plot_phonon_frequencies_converge_abs_widget)

## Phonon frequencies relative error (%)

Please check <a href="./sssp-docs.ipynb" target="_blank">Documentation</a> for more details.

In [ ]:
display(plot_phonon_frequencies_converge_rel_widget)

## Pressure absolute error (GPa)

Please check <a href="./sssp-docs.ipynb" target="_blank">Documentation</a> for more details.

In [ ]:
display(plot_pressure_converge_widget)

## Pressure relative error (GPa)

Please check <a href="./sssp-docs.ipynb" target="_blank">Documentation</a> for more details.

In [ ]:
display(plot_pressure_converge_rel_widget)

## $\Delta$-factor (meV)

Please check <a href="./sssp-docs.ipynb" target="_blank">Documentation</a> for more details.

In [ ]:
display(plot_delta_converge_widget)

## $\Delta$-factor relative error (%)

Please check <a href="./sssp-docs.ipynb" target="_blank">Documentation</a> for more details.

In [ ]:
display(plot_delta_converge_rel_widget)

## Bands distance (meV)

Please check <a href="./sssp-docs.ipynb" target="_blank">Documentation</a> for more details.

In [ ]:
display(plot_bands_converge_widget)